In [6]:
import cv2
import numpy as np
import math

class AngleCalculator:
    def __init__(self, image_path):
        # Read the image
        self.image = cv2.imread(image_path)
        self.original = self.image.copy()
        self.points = []
        self.angle_count = 0
        
    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONUP:
            # Store the point
            self.points.append((x, y))
            # Draw point marker
            cv2.circle(self.image, (x, y), 5, (0, 0, 255), -1)
            
            # If we have three points, calculate angle and prepare for next set
            if len(self.points) == 3:
                self.angle_count += 1
                self.calculate_and_display_angle()
                # Keep the image with the previous angle but clear points for next measurement
                self.points = []
            
            # Update display
            cv2.imshow('Image', self.image)
    
    def calculate_angle(self, p1, p2, p3):
        """Calculate angle between three points"""
        # Vectors from point 2 to points 1 and 3
        v1 = np.array([p1[0] - p2[0], p1[1] - p2[1]])
        v2 = np.array([p3[0] - p2[0], p3[1] - p2[1]])
        
        # Calculate angle using dot product
        cosine = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        angle = np.arccos(np.clip(cosine, -1.0, 1.0))
        return np.degrees(angle)
    
    def calculate_and_display_angle(self):
        """Calculate angle and draw it on image"""
        # Draw lines between points
        cv2.line(self.image, self.points[0], self.points[1], (255, 0, 0), 2)
        cv2.line(self.image, self.points[1], self.points[2], (255, 0, 0), 2)
        
        # Calculate angle
        angle = self.calculate_angle(self.points[0], self.points[1], self.points[2])
        
        # Display angle text with angle number
        text_pos = (self.points[1][0] + 10, self.points[1][1] - 10)
        cv2.putText(self.image, f"Angle {self.angle_count}: {angle:.1f}°", text_pos,
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    def run(self):
        # Create window and set mouse callback
        cv2.namedWindow('Image')
        cv2.setMouseCallback('Image', self.mouse_callback)
        
        while True:
            cv2.imshow('Image', self.image)
            key = cv2.waitKey(1) & 0xFF
            
            # Press 'r' to reset
            if key == ord('r'):
                self.image = self.original.copy()
                self.points = []
                self.angle_count = 0
            # Press 'q' to quit
            elif key == ord('q'):
                break
        
        cv2.destroyAllWindows()

# Usage
if __name__ == "__main__":
    calculator = AngleCalculator('.\img.png')
    calculator.run()